# Practical Exercise 3.06: Historical Simulation VaR

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd

#Download historical data

symbols = ['AAPL', 'AMZN', 'META', 'GOOGL', 'MSFT', 'NVDA', 'TSLA']
start = '2021-01-01'
end = None  # This will download data up to the present
data = yf.download(symbols, start=start, end=end, group_by='symbols', progress=False, auto_adjust=False, actions=False)

# Extract adjusted close prices

adj_close_data = data.xs('Adj Close', level=1, axis=1,)
adj_close_data= adj_close_data[symbols]

#Calculate daily returns for each asset

daily_returns = adj_close_data.pct_change(fill_method=None).dropna()
daily_returns = daily_returns.reindex(columns=symbols)

# Define portfolio weights (e.g., equal-weighted portfolio)

num_assets = len(symbols)  # Calculate the number of assets in the portfolio
weights = np.array([1.0 / num_assets] * num_assets)  # Equally weighted portfolio

# Calculate portfolio returns: Multiply each day's returns by the corresponding asset weights and sum them to get portfolio returns
portfolio_returns = daily_returns.dot(weights)

# Set the confidence level (e.g., 95% confidence level)

confidence_level = 0.95

# Sort the portfolio returns to identify worst-case losses

sorted_returns = np.sort(portfolio_returns)

# Calculate the VaR: VaR is the value at the (1-confidence_level) percentile of the sorted returns

var_percentile = (1 - confidence_level) * 100
VaR = np.percentile(sorted_returns, var_percentile)

# Calculate VaR in dollar terms based on the portfolio's total value

portfolio_value = 1_000_000  # Example portfolio value of $1 million
VaR_value = portfolio_value * -VaR  # VaR value (negative because it's a loss)

# Print the results

print(f"Historical Simulation VaR ({confidence_level*100}% confidence level): ${VaR_value:,.2f}")
